# 생성모델

- 학습 단계에선, Density estimation을 풀어내도록 학습
    - $p_{model}(x) ~ p_{data}(x)$가 되도록 'un-observable pdf($p_{data}(x)$)를 estimation!
    
    
- GAN : implicit density estimation
    - $p_{model}(x)$가 explicitly 정의되지 않음
    
    
- VAE : explicit density estimation
    - $p_{model}(x)$가 explicitly '근사(approximation)됨'
        - approximate density : intractable
            - cannot be directly optimized
                - So, Lower bound of likelihood optimize : maximizing the evidence lower bound(max ELBO)
    
    
- AR(Auto regressive) : explicit density estimation
    - $p_{model}(x)$가 explicitly '정의됨'
        - tractable
            - can be directly optimized
                - maximizing the likelihood of training data
                - 가능도 함수의 explicit modeling이 가능하다는 점에서 특징점

## AutoRegressive modeling을 위한 필요조건들

1. $x$ feature들에 대해 determining ordering(정해진 순서)을 가져야 함 (Auto regressive니까)
    - AR이 주로 시계열 데이터에 사용되는 이유
    - 시계열이 아닌 image 데이터에도 사용 가능! (임의로 순서지정)
        - left -> right
        - top -> bottom
        
    
2. $Joint PDF = \prod Conditional PDF$
    - $P(x) = \prod P(x_{i}|x_{1}, x_{2}, ..., x{i-1})$, i = determining order
    - joint modeling 문제를 'sequence problem'으로 치환
    - 핵심은 "$Conditional PDF$를 어덯게 얻어내느냐"
        - **PDF도 함수이다. Universial Approximator인 Deep NN을 사용해 PDF를 구하자.**

## PixelCNN

- Families of Auto-Regressive Generative model
- 순차적으로(Sequentially) PIXEL값을 생성하는 모델
- Convolution 연산으로 이미지 내 Pixel들의 distribution을 학습
    - Receptive field : 컨볼루션 연산으로 얻는 receptive field는 사실 sequential problem(pixel 단위)이라는 AR과 맞지 않음
        - 그래서 MASKing을 적용해, sequential하게 정보를 얻어갈 수 있도록 디자인!
            - 아직 예측되지 않은, 순서가 오지 않은 pixel 정보들을 차단!

<img src = "figure_2.jpeg" width="40%" height="40%">
        
- Masked convolutional layers
    - mask size == fileter size
    - mask type : A, B
        - **type A mask**
            - 첫 conv layer에만 적용됨(한 번만 사용되는 것)
            - 예측하려는 pixel의 위치를 '0'으로 mask!(컨볼루션 필터 연산이 적용되지 않음)
                - 예측하려는 위치 == filter와 mask의 center!
            - 첫 conv에 예측하려는 위치의 input 데이터를 보지 못하게하는 효과
            
        - **type B mask**
            - 이후 모든 conv layer에 적용됨
            - 예측하려는 pixel의 위치를 '1'로 설정(컨볼루션 필터 연산이 적용됨)
    
<img src = "figure_1.jpeg" width="80%" height="80%">


- Architecture

<img src = "figure_3.jpeg" width="80%" height="80%">

# PixelCNN

**Author:** [ADMoreau](https://github.com/ADMoreau)<br>
**Date created:** 2020/05/17<br>
**Last modified:** 2020/05/23<br>
**Description:** PixelCNN implemented in Keras.

## Introduction

PixelCNN is a generative model proposed in 2016 by van den Oord et al.
(reference: [Conditional Image Generation with PixelCNN Decoders](https://arxiv.org/abs/1606.05328)).
It is designed to generate images (or other data types) iteratively
from an input vector where the probability distribution of prior elements dictates the
probability distribution of later elements. In the following example, images are generated
in this fashion, pixel-by-pixel, via a masked convolution kernel that only looks at data
from previously generated pixels (origin at the top left) to generate later pixels.
During inference, the output of the network is used as a probability distribution
from which new pixel values are sampled to generate a new image
(here, with MNIST, the pixel values range from white (0) to black (255).


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F

from torch.autograd import Variable
from torch import cuda, backends
from torchvision.datasets import MNIST
from torchvision.utils import save_image
import torchvision.transforms as transforms

import numpy as np
import os
import time
from tqdm import tqdm

from tensorflow import keras

backends.cudnn.benchmark = True

## Getting the data

- 왜 33% value 이하는 0으로 하고, 33%로 정했는 가?

In [5]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)
n_residual_blocks = 5
batch_size = 12

# The data, split between train and test sets
download_root = './MNIST_DATASET'
train_dataset = MNIST(download_root, train=True, download=False)
test_dataset = MNIST(download_root, train=False, download=False)

x_train, x_test = train_dataset.data, test_dataset.data
x_train, x_test = torch.where(x_train < (.33*256), 0, 1), torch.where(x_test < (.33*256), 0, 1)

x_train = torch.cat([x_train, x_test], axis=0)
x_train = x_train.unsqueeze(dim=1)
x_train, x_val = data.random_split(x_train, [60000, 10000])

# print(f"x_train shape : {x_train.shape}, \nx_train type : {type(x_train)}, \nx_train unique val : {torch.unique(x_train)}")

train_loader = data.DataLoader(x_train, batch_size=batch_size, shuffle=True)
val_loader = data.DataLoader(x_val, batch_size=batch_size, shuffle=True)
print(f"train_loader # batchs : {len(train_loader)} \nval_loader # batchs   : {len(val_loader)}")

/home/jeongseobkim/anaconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


train_loader # batchs : 5000 
val_loader # batchs   : 834


## Create two classes for the requisite Layers for the model

- Mask type : 'A', 'B'

In [25]:
# The first layer is the PixelCNN layer. This layer simply
# builds on the 2D convolutional layer, but includes **masking**.
class PixelConvLayer(nn.Conv2d):
    
    def __init__(self, mask_type, *args, **kwargs):
        super().__init__(*args, **kwargs)
        assert mask_type in ('A', 'B')
        self.register_buffer('mask', self.weight.data.clone())
        _, _, kH, kW = self.weight.size()
        
        # 일단 1로 kernel 초기화
        self.mask.fill_(1)
        # mask_type==B : center pixel을 1로!, 나머지 오른쪽 column은 0으로!
        self.mask[:, :, kH // 2, kW // 2 + (mask_type == 'B'):] = 0
        # 가장 마지막 row를 0으로 채우기
        self.mask[:, :, kH // 2 + 1:] = 0

    def forward(self, x):
        self.weight.data *= self.mask
        return super(PixelConvLayer, self).forward(x)


# Next, we build our residual block layer.
# This is just a normal residual block, but **based on the PixelConvLayer**.
class ResidualBlock(nn.Module):
    
    def __init__(self, ch_in, ch_out, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.res_block = nn.Sequential(
            nn.Conv2d(ch_in, ch_out, kernel_size=1),
            nn.ReLU(inplace=False),
            
            PixelConvLayer(mask_type = 'B', in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=False),
            
            nn.Conv2d(ch_in, ch_out, kernel_size=1),
            nn.ReLU(inplace=False)
        )
        
    def forward(self, x):
        residual = x
        x = self.res_block(x)
        x += residual
        
        return x

**보충자료**

- ```self.register_buffer('mask', self.weight.data.clone())```
    - reference : https://discuss.pytorch.org/t/what-is-the-difference-between-register-buffer-and-register-parameter-of-nn-module/32723/7
    - ```self.register_buffer()``` : state_dict로 저장, 학습가능한 모델의 파라미터는 아니지만, 모델의 내부 정보로써 state_dict()로 호출 될 수 있는 정보로 저장하고 싶을 때 사용. model.parameters()에는 호출되지 않기 때문에, 학습(optimization)되진 않음
    

- ```state_dict() 란 무엇인가```
    - reference : https://tutorials.pytorch.kr/recipes/recipes/what_is_state_dict.html
    

In [24]:
# 보충자료 : self.register_buffer()
# reference : https://discuss.pytorch.org/t/what-is-the-difference-between-register-buffer-and-register-parameter-of-nn-module/32723/7

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.my_tensor = torch.randn(1)
        self.register_buffer('my_buffer', torch.randn(1))
        self.my_param = nn.Parameter(torch.randn(1))
        
    def forward(self, x):
            return x

model = MyModel()
print("model.my_tensor : ", model.my_tensor)
print("model.state_dict() : ", model.state_dict())

model.my_tensor :  tensor([1.4851])
model.state_dict() :  OrderedDict([('my_param', tensor([-1.0954])), ('my_buffer', tensor([-0.9836]))])


## Build the model based on the original paper


In [7]:
class PixelCNN(nn.Module):
    
    def __init__(self, ch_in, ch_out):
        super().__init__()
        self.in_conv = PixelConvLayer(mask_type='A', in_channels=ch_in, out_channels=ch_out, kernel_size=7, padding=3)
        self.res_blocks = nn.Sequential(*[ResidualBlock(ch_out, ch_out) for _ in range(n_residual_blocks)])
        self.out_conv = nn.Sequential(PixelConvLayer(mask_type='B', in_channels=ch_out, out_channels=ch_out, kernel_size=1),
                                      nn.ReLU(inplace=False),
                                      PixelConvLayer(mask_type='B', in_channels=ch_out, out_channels=ch_out, kernel_size=1),
                                      nn.ReLU(inplace=False))   
        self.out = nn.Sequential(
            nn.Conv2d(in_channels=ch_out, out_channels=ch_in, kernel_size=1),
            nn.Sigmoid()
        )
    
    
    def forward(self, x):
        
        x = self.in_conv(x)
        x = self.res_blocks(x)
        x = self.out_conv(x)
        x = self.out(x)
        
        return x        

In [8]:
model = PixelCNN(1, 128)

In [6]:
from torchinfo import summary

summary(model, input_size=(1, 1, 28, 28))

Layer (type:depth-idx)                        Output Shape              Param #
PixelCNN                                      --                        --
├─PixelConvLayer: 1-1                         [1, 128, 28, 28]          6,400
├─Sequential: 1-2                             [1, 128, 28, 28]          --
│    └─ResidualBlock: 2-1                     [1, 128, 28, 28]          --
│    │    └─Sequential: 3-1                   [1, 128, 28, 28]          180,608
│    └─ResidualBlock: 2-2                     [1, 128, 28, 28]          --
│    │    └─Sequential: 3-2                   [1, 128, 28, 28]          180,608
│    └─ResidualBlock: 2-3                     [1, 128, 28, 28]          --
│    │    └─Sequential: 3-3                   [1, 128, 28, 28]          180,608
│    └─ResidualBlock: 2-4                     [1, 128, 28, 28]          --
│    │    └─Sequential: 3-4                   [1, 128, 28, 28]          180,608
│    └─ResidualBlock: 2-5                     [1, 128, 28, 28]          

In [7]:
import torchsummary

torchsummary.summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
    PixelConvLayer-1          [-1, 128, 28, 28]           6,400
            Conv2d-2          [-1, 128, 28, 28]          16,512
              ReLU-3          [-1, 128, 28, 28]               0
    PixelConvLayer-4          [-1, 128, 28, 28]         147,584
              ReLU-5          [-1, 128, 28, 28]               0
            Conv2d-6          [-1, 128, 28, 28]          16,512
              ReLU-7          [-1, 128, 28, 28]               0
     ResidualBlock-8          [-1, 128, 28, 28]               0
            Conv2d-9          [-1, 128, 28, 28]          16,512
             ReLU-10          [-1, 128, 28, 28]               0
   PixelConvLayer-11          [-1, 128, 28, 28]         147,584
             ReLU-12          [-1, 128, 28, 28]               0
           Conv2d-13          [-1, 128, 28, 28]          16,512
             ReLU-14          [-1, 128,

## Training

In [8]:
optimizer = optim.Adam(model.parameters(), lr=0.0005)
criterion = nn.BCELoss()
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print("device :", device)

device : cuda:0


In [9]:
num_epochs = 50

errs_train, errs_val = [], []

for epoch in range(num_epochs):

    model.to(device)
    losses, losses_val = float(), float()

    time_tr = time.time()
    model.train()
    for x in train_loader:

        x = x.float().to(device)
        target = x.data.float().to(device)
        
        pred = model(x)

        loss = criterion(pred, target)
        
        losses += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    time_tr = time.time() - time_tr
    errs_train.append(losses/len(train_loader))
    

    time_ev = time.time()
    model.eval()
    with torch.no_grad():
        for x_val in val_loader:

            x_val = x_val.float().to(device)
            target_val = x_val.data.float().to(device)
            pred_val = model(x_val)

            loss_val = criterion(pred_val, target_val)
            
            losses_val += loss_val.item()
            
        time_ev = time.time() - time_ev
        errs_val.append(losses_val/len(val_loader))
        
    print(f"Epoch : {epoch}/{num_epochs} \ntrain loss : {errs_train[-1]:.4f}(sec : {time_tr:.1f}), val loss : {errs_val[-1]:.4f}(sec : {time_ev:.1f})")     


Epoch : 0/50 
train loss : 0.0932(sec : 36.4), val loss : 0.0872(sec : 1.8)
Epoch : 1/50 
train loss : 0.0862(sec : 36.4), val loss : 0.0857(sec : 1.9)
Epoch : 2/50 
train loss : 0.0848(sec : 36.5), val loss : 0.0842(sec : 1.9)
Epoch : 3/50 
train loss : 0.0840(sec : 36.7), val loss : 0.0839(sec : 1.9)
Epoch : 4/50 
train loss : 0.0834(sec : 36.7), val loss : 0.0835(sec : 1.9)
Epoch : 5/50 
train loss : 0.0830(sec : 38.5), val loss : 0.0832(sec : 1.9)
Epoch : 6/50 
train loss : 0.0827(sec : 38.6), val loss : 0.0833(sec : 1.9)
Epoch : 7/50 
train loss : 0.0824(sec : 37.0), val loss : 0.0830(sec : 1.9)
Epoch : 8/50 
train loss : 0.0822(sec : 37.0), val loss : 0.0830(sec : 1.9)
Epoch : 9/50 
train loss : 0.0820(sec : 37.3), val loss : 0.0828(sec : 2.0)
Epoch : 10/50 
train loss : 0.0818(sec : 39.6), val loss : 0.0827(sec : 2.0)
Epoch : 11/50 
train loss : 0.0816(sec : 39.0), val loss : 0.0829(sec : 1.9)
Epoch : 12/50 
train loss : 0.0815(sec : 37.1), val loss : 0.0827(sec : 1.9)
Epoch : 1

In [10]:
# torch.save(model.state_dict(), 'pixel_cnn.pth')

In [9]:
model = PixelCNN(1, 128)
model.load_state_dict(torch.load('pixel_cnn.pth'))
model.eval()

PixelCNN(
  (in_conv): PixelConvLayer(1, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (res_blocks): Sequential(
    (0): ResidualBlock(
      (res_block): Sequential(
        (0): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU()
        (2): PixelConvLayer(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU()
        (4): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
        (5): ReLU()
      )
    )
    (1): ResidualBlock(
      (res_block): Sequential(
        (0): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU()
        (2): PixelConvLayer(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU()
        (4): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
        (5): ReLU()
      )
    )
    (2): ResidualBlock(
      (res_block): Sequential(
        (0): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU()
        (2): PixelConvLayer(128, 128, k

In [13]:
# model.state_dict

In [13]:
del train_loader
del val_loader
del loss
del loss_val
torch.cuda.empty_cache()

## Demonstration

The PixelCNN cannot generate the full image at once. Instead, it must generate each pixel in
order, append the last generated pixel to the current image, and feed the image back into the
model to repeat the process.


In [20]:
from IPython.display import Image, display

# Create an empty array of pixels.
batch = 3
pixels = torch.zeros(size=(batch,) + (1, 28, 28))
batch, channels, rows, cols = pixels.shape

print(f"batch, channels, rows, cols : {batch, channels, rows, cols}")

# Iterate over the pixels because generation has to be done sequentially pixel by pixel.
model.eval()
with torch.no_grad():
    for row in tqdm(range(rows)):
        for col in range(cols):
            for channel in range(channels):

                # Feed the whole array and retrieving the pixel value probabilities for the next
                # pixel.
                model.to("cpu")   # because of cuda 'out of memory' error
                pred = model(pixels.float())
                probs = pred[:, channel, row, col]

                # Use the probabilities to pick pixel values and append the values to the image
                # frame.
                pixels[:, channel, row, col] = torch.ceil(
                    probs - torch.rand(size=probs.shape)
                )

batch, channels, rows, cols : (3, 1, 28, 28)


100%|██████████| 28/28 [00:13<00:00,  2.00it/s]


In [21]:
# Iterate over the generated images and plot them with matplotlib.
for i, pic in enumerate(pixels):
    pic = pic.detach().numpy()
    pic = torch.from_numpy(pic)
    save_image(pic, f"generated_img_{i}_torch.png")
    
    
display(Image("generated_img_0_torch.png"))
display(Image("generated_img_1_torch.png"))
display(Image("generated_img_2_torch.png"))